This notebook is to demonstrate the interface of the developed environment

Replace the variable *PATH_TO_ROOT* to run the notebook

In [1]:
import sys

PATH_TO_ROOT = 'C:/Users/walter/Desktop/git/AlphaBuilding-ResCommunity'
sys.path.insert(0,PATH_TO_ROOT)

In [2]:
import pandas as pd
import numpy as np

from gym_AlphaBuilding.envs import residential
from bin.util.distribution import utility
from bin.util.weather import noaa_weather

In [3]:
# Retrieve the Thermal Time Constant and Equivalent Temperature inferred from the Ecobee DYD Dataset 
# Users can use their own data or use the provided function (different value for different state)
city = 'Berkeley'
state = 'CA'
ttc = utility.get_ttc(state)
teq = utility.get_teq(state)

In [4]:
# Retrieve the comfort temperature zone using the provided function
# Four methods supported: 'ASHRAE PMV', 'ASHRAE adaptive', 'Wang2020', and 'ResStock'
tsp, trange = utility.get_comfort_temp('cooling', 'Wang2020')

In [5]:
SAMPLE_SIZE = 100
STEP_SIZE = 15 # min
start_date = '2018-07-01'
final_date = '2018-08-01'
year = 2018
sim_horizon = (start_date, final_date)

In [6]:
# Download weather data from NOAA weather stations
# If you want to use your own weather data, skip this step
address = '{}, {}'.format(city, state)
station_ids, station_names = noaa_weather.find_closest_weather_station(noaa_weather.geocode_address(address))

# You might need to try a couple of weather stations, because some weather stations have large missing rate 
weather = noaa_weather.download_weather(station_ids[0], year)
weather = weather.tz_convert('America/Los_Angeles').tz_localize(None)  # remove tz-awareness

# truncate and resample the weather data to fit the simulation horizon and time step
weather_h = weather.resample('1H').mean()  # hourlu average to remove noise
# ambient_temp = weather_h.resample('{}min'.format(STEP_SIZE)).interpolate()[['Temperature']]
weather_h = weather_h.resample('{}min'.format(STEP_SIZE)).interpolate()
weather_h = weather_h.truncate(*sim_horizon)

In [7]:
# Make the time index matches with simulation horizon and step size
weather_h = weather_h.resample('{}min'.format(STEP_SIZE)).interpolate()
weather_h.head()

,Temperature,CloudCoverage
DateTime,,
2018-07-01 00:00:00,14.400,4.0
2018-07-01 00:15:00,14.275,4.0
2018-07-01 00:30:00,14.150,4.0
2018-07-01 00:45:00,14.025,4.0
2018-07-01 01:00:00,13.900,4.0


In [8]:
# Set up the environment
env = residential.AlphaResEnv(sampleSize  = SAMPLE_SIZE, 
                           stepSize    = STEP_SIZE, 
                           simHorizon  = (start_date, final_date),
                           ambientWeather = weather_h, 
                           ttc = ttc, 
                           teq = teq, 
                           tsp = tsp, 
                           trange = trange,
                           hvacMode = 'cooling only')

C:\Users\walter\Anaconda3\envs\ecobee_env\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float16
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [14]:
# Get the parameter of the environment
env.getParameter()

,R,C,P_h,P_c,Q_h,Q_c,COP_h,COP_c,T_sp,T_range
0,3.178953,4.311134,5.503430,6.280656,13.892792,-17.489804,2.524388,2.784710,22.636747,1.212448
1,5.212068,5.266271,4.365370,2.427730,9.199646,-6.880872,2.107415,2.834282,22.716786,1.210666
2,2.879207,4.696964,6.909608,7.777653,16.125538,-21.138264,2.333785,2.717820,21.363123,1.209928
3,3.909824,3.438950,3.172327,3.729512,10.930604,-10.121846,3.445611,2.713987,22.880377,1.072514
4,4.904999,4.586961,4.030092,3.114177,7.844049,-7.703726,1.946370,2.473760,22.700973,1.145890
...,...,...,...,...,...,...,...,...,...,...
95,0.964838,2.199996,19.469290,13.034931,48.541267,-36.011679,2.493222,2.762706,21.824816,1.272785
96,3.707803,4.186959,5.297968,4.071871,13.759894,-10.519512,2.597202,2.583459,24.747277,1.200239
97,1.326314,6.278249,27.580173,19.093514,48.272835,-39.247956,1.750273,2.055565,23.387746,1.249595
98,1.418523,2.534950,15.677371,9.655946,34.892389,-28.923862,2.225653,2.995446,22.967204,1.353424


In [16]:
# Reset the environment to start simulation
obs = env.reset()
print(f'The length of the returned obs is {len(obs)}, whichs equals to 3 + {SAMPLE_SIZE}')

The length of the returned obs is 103, whichs equals to 3 + 100


In [17]:
# Select the control action: 1 for heating, 2 for cooling, 0 for free floating
acts = np.ones(100)*2

In [18]:
# One step simulation using the selected control action
obs, reward, done, comments = env.step(acts)

In [19]:
# Returned observations
obs

array([ 14.275     ,   4.        ,   0.        ,  21.799514  ,
        23.19061678,  19.62970293,  22.72365714,  22.32328225,
        19.13324538,  24.16810438,  20.85134337,  19.23294827,
        21.83197312,  20.96864367,  23.14936142,  21.29804592,
        20.59104784,  21.6651015 ,  19.93353615,  20.82738196,
        21.7308561 ,  21.64980244,  21.767025  ,  20.18778795,
        19.1655812 ,  22.83062735,  20.889484  ,  20.58653708,
        22.08616466,  21.93892298,  22.72706996,  22.43468837,
        23.16079656,  21.4775367 ,  21.30673446,  23.15865556,
        22.18800593,  21.96305539,  20.78172688,  22.00244483,
        23.2695205 ,  22.95420672,  21.1644798 ,  22.96447271,
        19.39806158,  22.39066632,  20.76205733,  23.10427196,
        20.95450402,  22.51536369,  23.99992726,  24.96457363,
        20.61700297,  20.76785318,  22.28681135,  23.29698025,
        22.27723211, -21.1601895 ,  22.05775195,  20.92683053,
        21.31429604,  22.29533252,  20.47757963,  22.51

In [20]:
## Returned comments, including
# Summed electricity consumption during the last time step (15min in this case), unit: kWh
# Summed uncomfortable degree hours during the last time step, unit: degC*h
# Internal heat gain for each TCL
# Solar heat gain for each TCL
# Modelling uncertainty for each TCL, unit: degC
# Measurement error for each TCL, unit: degC
comments

{'Energy': 236.48617559764037,
 'Comfort_UCDH': 30.60411613770671,
 'intHG': array([2.52028954, 4.17899077, 2.25803688, 0.59023725, 1.09778544,
        1.7647174 , 0.95084631, 3.94748221, 5.69286218, 0.75456931,
        1.49696049, 5.60348525, 2.39740774, 4.90005546, 0.16655678,
        0.83878269, 2.86929066, 2.88129307, 3.45702767, 5.10949322,
        2.98299978, 1.96623892, 3.49002733, 2.66451969, 4.08909779,
        0.97299659, 6.06233016, 2.39408864, 2.22092463, 4.05465292,
        3.68278545, 1.33164841, 4.14116974, 1.20296817, 4.77961981,
        1.84428249, 3.6262243 , 1.59414416, 0.81659635, 4.90371718,
        1.90734429, 3.42417355, 2.33195877, 0.49163445, 0.96038042,
        2.07351204, 1.06310272, 4.80880277, 2.29282951, 1.49889786,
        1.41198476, 2.09169426, 1.36491997, 5.0148112 , 2.61235064,
        2.35459997, 4.12222807, 0.71781037, 3.83218208, 0.02729512,
        2.15019237, 1.32086561, 2.51047428, 5.78111215, 2.70884299,
        3.37467906, 5.87103904, 4.333402

In [21]:
# Returned reward, caluculated as the weighted sum of energy consumption and uncomfortable degree hours using
# the costWeight input
# Users can use the observations and comments to re-calculate the rewards
reward

-542.5273369747075

In [22]:
# Flag indicating whether the eposide end or not
done

False